# Libraries and Imports

In [7]:
%pip install pydub


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import os

# adding the directory containing asr/asr_pipeline.py to the Python path
sys.path.append(os.path.abspath("../asr"))

In [8]:
from pydub import AudioSegment
from asr_pipeline import asr_infer_pipeline

# ASR
Section to load audio from disk and get transcript

In [10]:
def convert_to_wav(input_file: str, output_file: str) -> str:
    """
    Converts an audio file to .wav format.

    Args:
        input_file (str): The path to the input audio file.
        output_file (str): The path to the output .wav file.

    Returns:
        str: The path to the converted .wav file.
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_file)

    # Export as .wav
    audio.export(output_file, format="wav")

    return output_file

Covert audio file to wav format

In [11]:
input_file_path = "../ltl/test_samples/test01.m4a"  
output_file_path = "../ltl/test_samples/test01.wav"  

converted_file = convert_to_wav(input_file_path, output_file_path)



'../ltl/test_samples/test01.wav'

In [12]:
asr_infer_pipeline(converted_file) # transcribe file

'../ltl/test_samples/test01_transcription.json'